In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# move into project directory
repo_name = "UMass-CS670-Project"
%cd /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/$repo_name
!ls

/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project
common			  experiments		   last_run_info.json	  source-data
config.yaml		  get_dataset.sh	   models		  texture_params.txt
datautils		  github-repo-clone.ipynb  preprocess_dataset.py
demo			  Index.ipynb		   README.md
excluded-train-files.txt  index.py		   SINet.ipynb


In [ ]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [ ]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# uncomment the lines below  to download the data

import os

!pip install gdown

data_dir="source-data"

if not(os.path.exists("source-data")):
    #!gdown https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&export=download
    #echo 'Downloaded data! Unzipping to data folder'
    !unzip -qq -d . ./COD10K-v3.zip
    os.rename("./COD10K-v3.zip", "./source-data/COD10K-v3.zip")
else:
    print("\nThe data directory exists!")


The data directory exists!


In [ ]:
# setup some imports
from datautils.datareader import read_data
from datautils.dataset import COD10KDataset
from torch.utils.data import DataLoader
import random
import numpy as np
import torch
import argparse
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
import json

from experiments.style_transfer import style_transfer
from common.visualizer import layer_visualizer
from common.utils import init_config

In [ ]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
config_params = init_config()
print(config_params)

{'data_dir': '/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project/source-data', 'device': 'cuda', 'root_dir': '/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project', 'use_gpu': True}


In [ ]:
def run_style_transfer_pipeline(args, texture_name, style_weight, last_batch_run = -1):
    pos_data_paths = read_data('Train')

    dataset = COD10KDataset(pos_data_paths)

    dataloader = DataLoader(dataset, batch_size = args.batch_size, shuffle = False)

    for i_batch, batch in enumerate(dataloader):
        if i_batch > last_batch_run:
            print(f"Processing batch {i_batch}, image: {batch['img_name']} of dimensions: {batch['img'].shape}")
            style_img = read_image(f'./source-data/Textures/{texture_name}.jpg')
            new_img = style_transfer(
                batch['img'],
                style_img,
                [0, 2, 5, 14, 23],
                21,
                1e-4,
                [style_weight]*5,
                1e-5,
                args)
            img_name = batch['img_name'][0]
            img_name = img_name.replace(".jpg", "")
            img_name = f"./source-data/Train/Styled-Image/{img_name}-Texture-{texture_name}.jpg"
            save_image(new_img, img_name)
            last_run = {
                "last_batch": i_batch
            }
            print(f"completing transfer of img {i_batch} with texture {texture_name}")
            with open("./last_run_info.json", "w") as fp:
                json.dump(last_run, fp)

    print(f'Finished modifying train dataset images for {texture_name}')
    last_run = {
        "last_batch": -1
    }
    with open("./last_run_info.json", "w") as fp:
        json.dump(last_run, fp)
    return -1

In [ ]:

param_dict = {
    'moss': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'snow': {
        'epochs': 200,
        'style_weights': 1.5
    },
    'wet-sand': {
        'epochs': 200,
        'style_weights': 3.5
    },
    'sand-with-moss': {
        'epochs': 100,
        'style_weights': 2.5
    },
    'rain': {
        'epochs': 100,
        'style_weights': 4.5
    },
    'wood': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'grass': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'foliage-texture': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'blue-coral': {
        'epochs': 100,
        'style_weights': 1.1
    }
}

with open("./last_run_info.json") as fp:
    last_run = json.load(fp)["last_batch"]

selected_textures = ['moss', 'wet-sand', 'sand-with-moss', 'rain', 'foliage-texture']

for texture in selected_textures:

    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type = int, default = 1)
    parser.add_argument('--model_name', type=str, default='vgg')
    parser.add_argument('--lr', type=float, default=0.05)
    parser.add_argument('--max_iter', type=int, default=param_dict[texture]['epochs'])
    args = parser.parse_args(args=[])

    #style_img = read_image(f'./data/Textures/{texture}.jpg')
    #print(img.size())
    #layer_visualizer(img, args)

    run_style_transfer_pipeline(args, texture, param_dict[texture]['style_weights'], last_run)

In [ ]:
from datautils.datareader import read_data
from torch.utils.data import DataLoader
from datautils.dataset import COD10KDataset
from models.custom_models import get_model
from torchvision.transforms import transforms
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Subset
from random import shuffle
from tqdm import tqdm
import os
import random

from common.utils import get_config, save_experiment_output, save_model_helpers, load_model, get_modelinfo

def image_collate(batch):
    batchlist = list(map(list, zip(*batch)))
    return batchlist

def __convert_to_grascale(img):
    imin, imax = img.min(), img.max()
    x = (img - imin) / (imax - imin)
    return x

def __prepare_data(type = 'Train'):
    data_paths = read_data('Train', True)
    dataset = COD10KDataset(data_paths, 'binary_classification')
    smlen = int(0.1 * len(dataset))
    ridxs = random.sample(range(len(dataset)), smlen)
    smftr_dataset = Subset(dataset, ridxs)
    return dataset, smftr_dataset

def __get_experiment_chkpt(model, optimizer):

    cfg = get_config()
    root_dir = cfg["root_dir"]

    mpath = os.path.join(root_dir, "models/checkpoints/curr_model.pt")
    opath = os.path.join(root_dir, "models/checkpoints/curr_model_optimizer.pt")
    if os.path.exists(mpath):
        print("Loading saved model")
        saved_model = load_model(mpath)
        saved_optim = load_model(opath)
        #model_dict = saved_model["model_state"]
        model.load_state_dict(saved_model)
        optimizer.load_state_dict(saved_optim)
        model_info = get_modelinfo(True)
        return model, optimizer, model_info
    else:
        return model, optimizer, None

def __run_train_loop(args, model, optimizer, model_info, train_dataloader, val_dataloader, val_len):

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((args.resize_size, args.resize_size)),
        transforms.Lambda(__convert_to_grascale),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])
    ])

    loss_fn = torch.nn.BCEWithLogitsLoss()
    last_epoch = -1
    train_losses = []
    val_losses = []
    val_accs = []
    if model_info != None:
        last_epoch = model_info['last_epoch']
        train_losses, val_losses, val_accs = model_info['trlosshistory'], model_info['vallosshistory'], model_info['valacchistory']
        epoch_arr = list(range(last_epoch + 1, args.max_iter))
    else:
        epoch_arr = list(range(args.max_iter))

    for epoch_i in epoch_arr:
        avg_loss = 0
        print(f'Running epoch {epoch_i}')
        model.train()
        for i_batch, batch in enumerate(tqdm(train_dataloader, desc = '\tRunning through training set', position = 0, leave = True)):
            lbls = torch.tensor(batch[1]).type(torch.FloatTensor)
            #inds = torch.tensor(list(range(len(batch[1]))))
            #print(lbls.size(), lbls == 0)
            target = torch.zeros((len(batch[1]), 2))
            target[lbls == 1, 1] = 1.0
            target[lbls == 0, 0] = 0.0
            #print(target.size())
            #print(target)
            optimizer.zero_grad()
            #print(len(batch), type(batch[0]), len(batch[0]))
            img_batch = list(map(transform, batch[0]))
            img_batch = torch.stack(img_batch, 0)
            #print(img_batch.size())
            #tr_batch = transform(batch['img'])
            op = model(img_batch)
            loss = loss_fn(op, target)
            loss.backward()
            avg_loss += loss.item()
            optimizer.step()
            #print(op.size())
            #break

        train_losses.append(avg_loss / (i_batch + 1))

        with torch.no_grad():
            model.eval()
            val_acc = 0.0
            val_loss = 0.0
            for i_batch, batch in enumerate(tqdm(val_dataloader, desc = '\tRunning through training set', position = 0, leave = True)):
                lbls = torch.tensor(batch[1]).type(torch.FloatTensor)
                target = torch.zeros((len(batch[1]), 2))
                target[lbls == 1, 1] = 1.0
                target[lbls == 0, 0] = 0.0

                img_batch = list(map(transform, batch[0]))
                img_batch = torch.stack(img_batch, 0)
                #print(img_batch.size())
                #tr_batch = transform(batch['img'])
                op = model(img_batch)
                loss = loss_fn(op, target)
                lbls = torch.argmax(F.sigmoid(op), 1)
                #print(lbls, op, loss)
                val_acc += torch.eq(lbls, torch.tensor(batch[1])).sum()
                val_loss += loss.item()
            val_acc /= val_len
            val_loss /= (i_batch + 1)
            val_losses.append(val_loss)
            val_accs.append(val_acc.item())
        #print(train_losses, val_losses, val_accs)
        chkpt_info = {
            'trlosshistory': train_losses,
            'vallosshistory': val_losses,
            'valacchistory': val_accs,
            'last_epoch': epoch_i
        }
        save_experiment_output(model, chkpt_info)
        save_model_helpers(optimizer.state_dict())
    save_experiment_output(model, chkpt_info, False)
    save_model_helpers(optimizer.state_dict(), False)
    print('\n\n')
    plt.plot(list(range(args.max_iter)), train_losses, color = 'red')
    plt.plot(list(range(args.max_iter)), val_losses, color = 'blue')
    plt.legend(['Training loss', 'Validation loss'])
    plt.title('Loss - Epoch plot')
    plt.show()
    print('\n\n\n')
    plt.clf()
    plt.plot(list(range(args.max_iter)), val_accs)
    plt.title('Validation accuracy plot')
    plt.show()


def binary_classification(args):
    full_dataset, smftr_dataset = __prepare_data()
    fulllen = len(full_dataset)
    smlen = len(smftr_dataset)
    idxs = list(range(fulllen))
    vlen = int(0.2 * fulllen)
    val_idxs = idxs[:vlen]
    tr_idxs = idxs[vlen:]
    train_dataset = Subset(full_dataset, tr_idxs)
    val_dataset = Subset(full_dataset, val_idxs)
    #train_dataloader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle = False, collate_fn = image_collate)
    train_dataloader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle = False, collate_fn = image_collate)
    val_dataloader = DataLoader(val_dataset, batch_size = args.batch_size, shuffle = False, collate_fn = image_collate)
    model = get_model(2, args.model_name)

    optimizer = torch.optim.Adam(model.parameters(), lr = args.lr, betas = (0.99, 0.999))

    model, optimizer, model_info = __get_experiment_chkpt(model, optimizer)

    __run_train_loop(args, model, optimizer, model_info, train_dataloader, val_dataloader, vlen)

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type = int, default = 64)
parser.add_argument('--model_name', type=str, default='resnet18')
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--max_iter', type=int, default=5)
parser.add_argument('--resize_size', type=int, default=336)
args = parser.parse_args(args=[])

binary_classification(args)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


Running epoch 0


	Running through training set:  95%|█████████▍| 71/75 [20:40<03:07, 46.83s/it]